## 1. Imports y Configuración

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
from tqdm.auto import tqdm
import time
from dtaidistance import dtw

# Configurar paths
project_root = Path.cwd().parent.parent
data_dir = project_root / 'data'
output_dir = data_dir / 'outputs' / 'cache'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Directorio de proyecto: {project_root}")
print(f"📁 Directorio de datos: {data_dir}")
print(f"📁 Directorio de cache: {output_dir}")

📁 Directorio de proyecto: /Users/mkurno/Documents/GitHub/ST
📁 Directorio de datos: /Users/mkurno/Documents/GitHub/ST/data
📁 Directorio de cache: /Users/mkurno/Documents/GitHub/ST/data/outputs/cache


## 3. Funciones DTW y Análisis de Gaps

## 2. Configuración de Filtro Temporal (OPCIONAL)

In [2]:
# ============================================================================
# FILTRO TEMPORAL: Configurá el rango de años para el análisis DTW
# ============================================================================
# 
# Dejá estos valores en None para usar TODOS los años disponibles (comportamiento default)
# O especificá un rango de años para filtrar solo países con cobertura completa en ese período
#
# Podés especificar:
#   - Ambos valores: rango completo definido
#   - Solo start: desde ese año hasta el máximo disponible
#   - Solo end: desde el mínimo disponible hasta ese año
#   - Ambos None: sin filtro (todos los años)
#
# Ejemplos:
#   year_filter_start = None      # Sin filtro (comportamiento original)
#   year_filter_end = None
#
#   year_filter_start = 1981      # Solo países con datos completos 1981-2021
#   year_filter_end = 2021        # Resultado: ~122 países
#
#   year_filter_start = 2010      # Solo países con datos desde 2010 en adelante
#   year_filter_end = None        # Resultado: ~122 países
#
# ============================================================================

year_filter_start = 1989  # 👈 Cambiar aquí: None o año inicial (ej: 1981)
year_filter_end = 2021    # 👈 Cambiar aquí: None o año final (ej: 2021)

# Validar configuración
if year_filter_start is not None and year_filter_end is not None:
    # Ambos valores especificados: validar que start <= end
    if year_filter_start > year_filter_end:
        raise ValueError(f"❌ Error: year_filter_start ({year_filter_start}) no puede ser mayor que year_filter_end ({year_filter_end})")
    print("🔍 FILTRO TEMPORAL ACTIVADO (rango completo)")
    print("="*70)
    print(f"   📅 Rango de años: {year_filter_start} - {year_filter_end}")
    print(f"   📊 Solo se incluirán países con datos COMPLETOS en este período")
    print("="*70)

elif year_filter_start is not None:
    # Solo start especificado: desde start hasta el máximo disponible
    print("🔍 FILTRO TEMPORAL ACTIVADO (desde año específico)")
    print("="*70)
    print(f"   📅 Desde: {year_filter_start} hasta el año máximo disponible")
    print(f"   📊 Solo se incluirán países con datos desde {year_filter_start} en adelante")
    print("="*70)

elif year_filter_end is not None:
    # Solo end especificado: desde el mínimo disponible hasta end
    print("🔍 FILTRO TEMPORAL ACTIVADO (hasta año específico)")
    print("="*70)
    print(f"   📅 Hasta: {year_filter_end} desde el año mínimo disponible")
    print(f"   📊 Solo se incluirán países con datos hasta {year_filter_end}")
    print("="*70)

else:
    # Ambos None: sin filtro
    print("✅ Sin filtro temporal - Se usarán TODOS los años disponibles")

🔍 FILTRO TEMPORAL ACTIVADO (rango completo)
   📅 Rango de años: 1989 - 2021
   📊 Solo se incluirán países con datos COMPLETOS en este período


In [3]:
def calculate_basic_dtw_with_path(series1, series2):
    """
    Calcula DTW entre dos series temporales y retorna distancia y path.
    Solo se usa en la celda opcional de paths.
    """
    n, m = len(series1), len(series2)
    
    dtw_matrix = np.full((n + 1, m + 1), np.inf)
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = abs(series1[i-1] - series2[j-1])
            dtw_matrix[i, j] = cost + min(
                dtw_matrix[i-1, j],
                dtw_matrix[i, j-1],
                dtw_matrix[i-1, j-1]
            )
    
    path = []
    i, j = n, m
    
    while i > 0 and j > 0:
        path.append((i-1, j-1))
        candidates = [
            (dtw_matrix[i-1, j-1], i-1, j-1),
            (dtw_matrix[i-1, j], i-1, j),
            (dtw_matrix[i, j-1], i, j-1)
        ]
        _, i, j = min(candidates, key=lambda x: x[0])
    
    path.reverse()
    return dtw_matrix[n, m], path

In [4]:
def analyze_gaps_in_series(df, country_code):
    """
    Analiza gaps (años sin datos) en la serie temporal de un país.
    Retorna información sobre continuidad de la serie.
    """
    country_data = df[df['country_code'] == country_code].sort_values('year')
    
    if len(country_data) == 0:
        return None
    
    years = country_data['year'].values
    first_year = years[0]
    last_year = years[-1]
    expected_years = last_year - first_year + 1
    actual_years = len(years)
    gaps = expected_years - actual_years
    
    # Encontrar los años faltantes
    all_years = set(range(first_year, last_year + 1))
    present_years = set(years)
    missing_years = sorted(all_years - present_years)
    
    # Encontrar segmentos continuos
    segments = []
    if len(years) > 0:
        segment_start = years[0]
        prev_year = years[0]
        
        for year in years[1:]:
            if year != prev_year + 1:  # Hay un gap
                segments.append((segment_start, prev_year, prev_year - segment_start + 1))
                segment_start = year
            prev_year = year
        
        # Agregar último segmento
        segments.append((segment_start, prev_year, prev_year - segment_start + 1))
    
    # Encontrar el segmento continuo más largo
    longest_segment = max(segments, key=lambda x: x[2]) if segments else None
    
    return {
        'country_code': country_code,
        'first_year': first_year,
        'last_year': last_year,
        'years_span': expected_years,
        'years_with_data': actual_years,
        'gaps': gaps,
        'missing_years': missing_years,
        'segments': segments,
        'longest_segment': longest_segment,
        'longest_segment_length': longest_segment[2] if longest_segment else 0,
        'is_continuous': gaps == 0
    }

print("✅ Función analyze_gaps_in_series definida")

✅ Función analyze_gaps_in_series definida


## 3. Cargar y Filtrar Datos GNI

Cargamos desde `todos_los_datos.csv` y filtramos igual que el notebook principal:
- Países con ≥20 años de datos GNI
- Solo países SIN gaps (series continuas)

In [5]:
# Cargar dataset principal (igual que notebook principal)
indicadores_file = data_dir / 'indicadores' / 'todos_los_datos.csv'
print(f"📊 Cargando datos desde: {indicadores_file}")
df_indicadores = pd.read_csv(indicadores_file, index_col=0)

print(f"   ✅ Datos cargados: {df_indicadores.shape}")
print(f"   📅 Rango años: {df_indicadores['year'].min()} - {df_indicadores['year'].max()}")
print(f"   🌍 Países totales: {df_indicadores['country_code'].nunique()}")

# Extraer datos de GNI
print(f"\n💰 Filtrando datos de GNI...")
gni_data = df_indicadores[['country_code', 'country_name', 'year', 'gni']].copy()

# PASO 1: Filtrar países con suficientes años de datos (≥20 años)
print(f"\n🔍 PASO 1: Filtrar países con datos suficientes...")
min_years_required = 20

countries_with_sufficient_gni = []
for country_code in gni_data['country_code'].unique():
    country_gni = gni_data[gni_data['country_code'] == country_code]
    valid_gni_years = country_gni['gni'].notna().sum()
    
    if valid_gni_years >= min_years_required:
        countries_with_sufficient_gni.append(country_code)

print(f"   ✅ Países con ≥{min_years_required} años: {len(countries_with_sufficient_gni)}")

# Filtrar dataset
gni_data = gni_data[gni_data['country_code'].isin(countries_with_sufficient_gni)].copy()
gni_data = gni_data.dropna(subset=['gni'])

print(f"   📊 Dataset GNI filtrado: {gni_data.shape}")
print(f"   🌍 Países a analizar: {gni_data['country_code'].nunique()}")

📊 Cargando datos desde: /Users/mkurno/Documents/GitHub/ST/data/indicadores/todos_los_datos.csv
   ✅ Datos cargados: (13020, 24)
   📅 Rango años: 1963 - 2022
   🌍 Países totales: 217

💰 Filtrando datos de GNI...

🔍 PASO 1: Filtrar países con datos suficientes...
   ✅ Países con ≥20 años: 197
   📊 Dataset GNI filtrado: (9295, 4)
   🌍 Países a analizar: 197


In [6]:
# PASO 2: Analizar gaps y excluir países con gaps (igual que notebook principal)
print(f"\n🔍 PASO 2: Analizando gaps en series temporales...")
print("="*70)

gap_analysis = []
for country_code in gni_data['country_code'].unique():
    analysis = analyze_gaps_in_series(gni_data, country_code)
    if analysis:
        gap_analysis.append(analysis)

gap_df = pd.DataFrame(gap_analysis)

# Estadísticas
total_countries = len(gap_df)
continuous_countries = gap_df['is_continuous'].sum()
countries_with_gaps = total_countries - continuous_countries

print(f"📈 ESTADÍSTICAS DE CONTINUIDAD:")
print(f"   Total de países: {total_countries}")
print(f"   Países con series continuas (sin gaps): {continuous_countries}")
print(f"   Países con gaps: {countries_with_gaps}")

# Mostrar países con gaps
if countries_with_gaps > 0:
    print(f"\n⚠️  PAÍSES CON GAPS A EXCLUIR:")
    countries_with_gaps_list = gap_df[gap_df['gaps'] > 0].sort_values('gaps', ascending=False)
    for _, row in countries_with_gaps_list.iterrows():
        print(f"   • {row['country_code']:5s}: {row['gaps']:2d} gaps de {row['years_span']:2d} años "
              f"({row['first_year']}-{row['last_year']})")

# EXCLUIR países con gaps
print(f"\n🚫 Excluyendo países con gaps del análisis DTW...")
countries_with_gaps_codes = gap_df[gap_df['gaps'] > 0]['country_code'].tolist()

countries_before = gni_data['country_code'].nunique()
gni_data = gni_data[~gni_data['country_code'].isin(countries_with_gaps_codes)].copy()
countries_after = gni_data['country_code'].nunique()

print(f"   ✅ Países antes: {countries_before}")
print(f"   ✅ Países después: {countries_after}")
print(f"   🗑️  Países excluidos: {countries_before - countries_after}")
print(f"\n✅ Dataset inicial: {countries_after} países con series continuas (sin gaps)")

# ============================================================================
# PASO 3: Aplicar filtro temporal si está configurado
# ============================================================================
if year_filter_start is not None or year_filter_end is not None:
    # Determinar rango efectivo (usar min/max disponibles si no se especificaron)
    available_min_year = gni_data['year'].min()
    available_max_year = gni_data['year'].max()
    
    effective_start = year_filter_start if year_filter_start is not None else available_min_year
    effective_end = year_filter_end if year_filter_end is not None else available_max_year
    
    print(f"\n🔍 PASO 3: Aplicando filtro temporal {effective_start}-{effective_end}...")
    print("="*70)
    
    # Para cada país, verificar si tiene datos completos en el rango especificado
    countries_with_complete_coverage = []
    
    for country_code in gni_data['country_code'].unique():
        country_data = gni_data[
            (gni_data['country_code'] == country_code) &
            (gni_data['year'] >= effective_start) &
            (gni_data['year'] <= effective_end)
        ]
        
        # Verificar cobertura completa
        years_needed = effective_end - effective_start + 1
        years_available = country_data['year'].nunique()
        
        if years_available == years_needed:
            countries_with_complete_coverage.append(country_code)
    
    # Filtrar dataset a solo esos países y ese rango de años
    countries_before_filter = gni_data['country_code'].nunique()
    
    gni_data = gni_data[
        (gni_data['country_code'].isin(countries_with_complete_coverage)) &
        (gni_data['year'] >= effective_start) &
        (gni_data['year'] <= effective_end)
    ].copy()
    
    countries_after_filter = gni_data['country_code'].nunique()
    
    print(f"   📊 Países con cobertura completa en {effective_start}-{effective_end}: {countries_after_filter}")
    print(f"   🗑️  Países excluidos por filtro temporal: {countries_before_filter - countries_after_filter}")
    print(f"   📅 Años en el dataset: {gni_data['year'].min()} - {gni_data['year'].max()}")
    print(f"\n✅ Dataset filtrado: {countries_after_filter} países × {effective_end - effective_start + 1} años")
else:
    print(f"\n✅ Sin filtro temporal - Usando todos los años disponibles")

print("="*70)


🔍 PASO 2: Analizando gaps en series temporales...
📈 ESTADÍSTICAS DE CONTINUIDAD:
   Total de países: 197
   Países con series continuas (sin gaps): 185
   Países con gaps: 12

⚠️  PAÍSES CON GAPS A EXCLUIR:
   • GRC  : 36 gaps de 60 años (1963-2022)
   • AFG  : 26 gaps de 59 años (1963-2021)
   • CHE  : 25 gaps de 60 años (1963-2022)
   • SOM  : 22 gaps de 60 años (1963-2022)
   • KHM  : 20 gaps de 60 años (1963-2022)
   • SWZ  : 15 gaps de 60 años (1963-2022)
   • CHI  : 12 gaps de 38 años (1970-2007)
   • FRO  : 10 gaps de 58 años (1965-2022)
   • WSM  : 10 gaps de 41 años (1982-2022)
   • FSM  :  9 gaps de 40 años (1983-2022)
   • GNQ  :  2 gaps de 60 años (1963-2022)
   • IRN  :  2 gaps de 60 años (1963-2022)

🚫 Excluyendo países con gaps del análisis DTW...
📈 ESTADÍSTICAS DE CONTINUIDAD:
   Total de países: 197
   Países con series continuas (sin gaps): 185
   Países con gaps: 12

⚠️  PAÍSES CON GAPS A EXCLUIR:
   • GRC  : 36 gaps de 60 años (1963-2022)
   • AFG  : 26 gaps de 59 

## 4. Preparar Series Temporales

Para cada país, guardamos solo los años donde tiene datos reales (sin relleno artificial).
Cada comparación DTW usará solo el período superpuesto entre los dos países (o el rango filtrado si se especificó).

### 🤔 Nota Metodológica: ¿Por qué alineamiento temporal?

**DTW puede comparar series de diferente longitud**, pero en este análisis elegimos **extraer períodos superpuestos** antes de calcular DTW. ¿Por qué?

#### Dos enfoques posibles:

**1. DTW Puro (sin alineamiento temporal):**
- Usa las series completas de cada país, sin importar longitud
- Argentina: 40 puntos (1983-2022) vs Uruguay: 60 puntos (1963-2022)
- DTW encuentra el mejor alineamiento entre patrones
- ⚠️ **Problema:** Puede alinear diferentes períodos históricos (ej: crisis 1970 de Uruguay con crisis 1990 de Argentina)

**2. Alineamiento Temporal + DTW (enfoque actual):**
- Extrae los mismos años calendario de ambos países
- Argentina: 40 puntos (1983-2022) vs Uruguay: 40 puntos (1983-2022)
- DTW compara el mismo período histórico
- ✅ **Ventaja:** Garantiza interpretación económica correcta (mismo año = mismo contexto global)

#### Decisión de diseño:

En análisis económico, **el contexto temporal importa**. La crisis de 2001 debe compararse con 2001, no con 1970. Por eso usamos alineamiento temporal: extraemos solo años superpuestos y luego aplicamos DTW.

**Resultado:** Cada par de países se compara usando solo los años donde ambos tienen datos, preservando la correspondencia temporal.

In [7]:
# Crear series temporales de GNI (solo años con datos reales, sin relleno)
print(f"\n📈 Creando series temporales de GNI...")
print("="*70)

# Obtener lista de países finales (sin gaps)
countries = sorted(gni_data['country_code'].unique())
print(f"   🌍 Países a procesar: {len(countries)}")

# Crear diccionarios con series de longitud variable (solo datos reales)
gni_raw = {}
gni_standardized = {}
country_years = {}  # Para saber qué años tiene cada país

countries_processed = 0

for country_code in countries:
    country_data = gni_data[gni_data['country_code'] == country_code].sort_values('year')
    
    # Extraer años y valores (solo datos reales, SIN relleno)
    years = country_data['year'].values
    gni_values = country_data['gni'].values
    
    # Guardar años para este país
    country_years[country_code] = years
    
    # Serie raw
    gni_raw[country_code] = np.array(gni_values, dtype=float)
    
    # Serie estandarizada (Z-score)
    if len(gni_values) > 1 and np.std(gni_values) > 0:
        gni_mean = np.mean(gni_values)
        gni_std = np.std(gni_values)
        gni_standardized[country_code] = (gni_values - gni_mean) / gni_std
    else:
        gni_standardized[country_code] = np.zeros_like(gni_values, dtype=float)
    
    countries_processed += 1
    if countries_processed % 20 == 0:
        print(f"   Procesados: {countries_processed}/{len(countries)} países")

print(f"\n✅ Series temporales creadas (longitud variable):")
print(f"   📊 Países procesados: {len(gni_raw)}")

# Mostrar algunos ejemplos de longitud de series
print(f"\n📊 Ejemplos de longitud de series:")
example_countries = ['ARG', 'FRA', 'BRA', 'RUS', 'LBY'] if all(c in gni_raw for c in ['ARG', 'FRA', 'BRA', 'RUS', 'LBY']) else list(gni_raw.keys())[:5]
for country in example_countries[:5]:
    if country in gni_raw:
        years = country_years[country]
        print(f"   {country}: {len(gni_raw[country])} puntos ({years[0]}-{years[-1]})")

print("="*70)


📈 Creando series temporales de GNI...
   🌍 Países a procesar: 141
   Procesados: 20/141 países
   Procesados: 40/141 países
   Procesados: 60/141 países
   Procesados: 80/141 países
   Procesados: 100/141 países
   Procesados: 120/141 países
   Procesados: 140/141 países

✅ Series temporales creadas (longitud variable):
   📊 Países procesados: 141

📊 Ejemplos de longitud de series:
   ABW: 33 puntos (1989-2021)
   AGO: 33 puntos (1989-2021)
   ALB: 33 puntos (1989-2021)
   ARG: 33 puntos (1989-2021)
   ATG: 33 puntos (1989-2021)


## 5. Calcular Matriz de Distancias DTW (RÁPIDO)

In [8]:
countries = sorted(gni_standardized.keys())
total_comparisons = len(countries) * (len(countries) - 1) // 2

print(f"🔄 Calculando matriz de distancias DTW (usando períodos superpuestos)...")
print(f"   Total de países: {len(countries)}")
print(f"   Total de comparaciones: {total_comparisons:,}")
print(f"\n⚡ Usando dtw.distance() con períodos donde ambos países tienen datos\n")

distances_data = []
start_time = time.time()

# Contadores para estadísticas
no_overlap_count = 0
min_overlap = float('inf')
max_overlap = 0

with tqdm(total=total_comparisons, desc="Calculando DTW") as pbar:
    for i, country1 in enumerate(countries):
        series1_full = gni_standardized[country1]
        years1 = country_years[country1]
        
        for country2 in countries[i+1:]:
            series2_full = gni_standardized[country2]
            years2 = country_years[country2]
            
            # Encontrar años superpuestos
            overlap_years = np.intersect1d(years1, years2)
            overlap_count = len(overlap_years)
            
            if overlap_count == 0:
                # Sin superposición: distancia infinita
                dtw_distance = np.inf
                no_overlap_count += 1
            else:
                # Extraer solo los años superpuestos
                idx1 = np.isin(years1, overlap_years)
                idx2 = np.isin(years2, overlap_years)
                
                series1_overlap = series1_full[idx1]
                series2_overlap = series2_full[idx2]
                
                # Calcular DTW solo con datos superpuestos
                dtw_distance = dtw.distance(series1_overlap, series2_overlap)
                
                # Actualizar estadísticas
                min_overlap = min(min_overlap, overlap_count)
                max_overlap = max(max_overlap, overlap_count)
            
            distances_data.append({
                'country1': country1,
                'country2': country2,
                'dtw_distance': dtw_distance,
                'overlap_years': overlap_count  # Guardar cuántos años se superponen
            })
            
            pbar.update(1)

elapsed_time = time.time() - start_time
print(f"\n✅ Cálculo completado en {elapsed_time/60:.2f} minutos")
print(f"   Total de distancias calculadas: {len(distances_data):,}")
print(f"   Velocidad promedio: {total_comparisons/elapsed_time:.0f} comparaciones/segundo")

print(f"\n📊 Estadísticas de superposición:")
print(f"   Pares sin superposición: {no_overlap_count}")
if min_overlap != float('inf'):
    print(f"   Mínima superposición: {min_overlap} años")
    print(f"   Máxima superposición: {max_overlap} años")

🔄 Calculando matriz de distancias DTW (usando períodos superpuestos)...
   Total de países: 141
   Total de comparaciones: 9,870

⚡ Usando dtw.distance() con períodos donde ambos países tienen datos



Calculando DTW:   0%|          | 0/9870 [00:00<?, ?it/s]


✅ Cálculo completado en 0.12 minutos
   Total de distancias calculadas: 9,870
   Velocidad promedio: 1353 comparaciones/segundo

📊 Estadísticas de superposición:
   Pares sin superposición: 0
   Mínima superposición: 33 años
   Máxima superposición: 33 años


## 6. Guardar Resultados

In [9]:
distances_df = pd.DataFrame(distances_data)

distances_file = output_dir / 'dtw_distances_matrix.csv'
distances_df.to_csv(distances_file, index=False)
print(f"💾 Distancias guardadas en: {distances_file}")
print(f"   Tamaño del archivo: {distances_file.stat().st_size / 1024:.1f} KB")
print(f"   Columnas: {list(distances_df.columns)}")
print(f"\n📝 Nota: La columna 'overlap_years' indica cuántos años se superponen entre cada par.")
print(f"   Los paths de alineación no están incluidos en este cache rápido.")
print(f"   Si necesitas los paths, ejecuta la celda opcional a continuación.")

💾 Distancias guardadas en: /Users/mkurno/Documents/GitHub/ST/data/outputs/cache/dtw_distances_matrix.csv
   Tamaño del archivo: 286.7 KB
   Columnas: ['country1', 'country2', 'dtw_distance', 'overlap_years']

📝 Nota: La columna 'overlap_years' indica cuántos años se superponen entre cada par.
   Los paths de alineación no están incluidos en este cache rápido.
   Si necesitas los paths, ejecuta la celda opcional a continuación.


## 6.1 [OPCIONAL] Calcular Paths de Alineación

⚠️ **Solo ejecuta esta celda si necesitas los paths de alineación DTW**

Este cálculo es **mucho más lento** (puede tomar 30-60 minutos).

In [10]:
# CELDA OPCIONAL: Calcular paths (lento)
print("⚠️  Calculando paths de alineación (esto tomará más tiempo)...\n")

paths_dict = {}
start_time_paths = time.time()

with tqdm(total=total_comparisons, desc="Calculando paths DTW") as pbar:
    for i, country1 in enumerate(countries):
        series1 = gni_standardized[country1]
        
        for country2 in countries[i+1:]:
            series2 = gni_standardized[country2]
            
            _, dtw_path = calculate_basic_dtw_with_path(series1, series2)
            
            paths_dict[(country1, country2)] = dtw_path
            paths_dict[(country2, country1)] = [(j, i) for i, j in dtw_path]
            
            pbar.update(1)

elapsed_paths = time.time() - start_time_paths

paths_file = output_dir / 'dtw_paths.pkl'
with open(paths_file, 'wb') as f:
    pickle.dump(paths_dict, f)

print(f"\n✅ Paths calculados en {elapsed_paths/60:.2f} minutos")
print(f"💾 Paths guardados en: {paths_file}")
print(f"   Tamaño: {paths_file.stat().st_size / 1024:.1f} KB")
print(f"   Total de paths: {len(paths_dict):,}")

⚠️  Calculando paths de alineación (esto tomará más tiempo)...



Calculando paths DTW:   0%|          | 0/9870 [00:00<?, ?it/s]


✅ Paths calculados en 0.08 minutos
💾 Paths guardados en: /Users/mkurno/Documents/GitHub/ST/data/outputs/cache/dtw_paths.pkl
   Tamaño: 5457.5 KB
   Total de paths: 19,740


## 7. Verificación y Estadísticas

In [11]:
# Filtrar pares con superposición válida (no infinitos)
valid_distances = distances_df[~np.isinf(distances_df['dtw_distance'])].copy()

print("\n📊 Estadísticas de distancias DTW (solo pares con superposición):\n")
print(f"   Pares válidos: {len(valid_distances)} de {len(distances_df)}")
print(f"   Mínima: {valid_distances['dtw_distance'].min():.2f}")
print(f"   Máxima: {valid_distances['dtw_distance'].max():.2f}")
print(f"   Media: {valid_distances['dtw_distance'].mean():.2f}")
print(f"   Mediana: {valid_distances['dtw_distance'].median():.2f}")

print(f"\n🔍 Top 10 pares más similares (menor distancia):")
top_similar = valid_distances.nsmallest(10, 'dtw_distance')[['country1', 'country2', 'dtw_distance', 'overlap_years']]
print(top_similar.to_string(index=False))

print(f"\n🔍 Top 10 pares más diferentes (mayor distancia):")
top_different = valid_distances.nlargest(10, 'dtw_distance')[['country1', 'country2', 'dtw_distance', 'overlap_years']]
print(top_different.to_string(index=False))

# Estadísticas de superposición
print(f"\n📊 Estadísticas de superposición temporal:")
print(f"   Media de años superpuestos: {valid_distances['overlap_years'].mean():.1f}")
print(f"   Mediana de años superpuestos: {valid_distances['overlap_years'].median():.0f}")
print(f"   Mínimo: {valid_distances['overlap_years'].min()} años")
print(f"   Máximo: {valid_distances['overlap_years'].max()} años")


📊 Estadísticas de distancias DTW (solo pares con superposición):

   Pares válidos: 9870 de 9870
   Mínima: 0.17
   Máxima: 5.42
   Media: 1.50
   Mediana: 1.33

🔍 Top 10 pares más similares (menor distancia):
country1 country2  dtw_distance  overlap_years
     AUT      BEL      0.172331             33
     BTN      IND      0.298473             33
     BEL      DNK      0.336509             33
     CRI      PHL      0.338721             33
     BOL      PAN      0.340471             33
     JOR      PHL      0.340705             33
     GTM      TZA      0.349077             33
     CRI      JOR      0.354860             33
     OMN      QAT      0.358819             33
     BOL      BTN      0.378857             33

🔍 Top 10 pares más diferentes (mayor distancia):
country1 country2  dtw_distance  overlap_years
     ETH      JPN      5.419853             33
     BGD      JPN      5.329239             33
     JPN      ZWE      5.305420             33
     BGD      SDN      5.299921   

## 8. Análisis Específico: Argentina

In [12]:
# Buscar comparaciones con Argentina usando el código de país 'ARG'
argentina_comparisons = distances_df[
    (distances_df['country1'] == 'ARG') | (distances_df['country2'] == 'ARG')
].copy()

# Normalizar para que Argentina siempre esté en country1
mask = argentina_comparisons['country2'] == 'ARG'
argentina_comparisons.loc[mask, ['country1', 'country2']] = argentina_comparisons.loc[mask, ['country2', 'country1']].values

# Filtrar solo comparaciones válidas (con superposición)
argentina_valid = argentina_comparisons[~np.isinf(argentina_comparisons['dtw_distance'])].copy()

print("\n🇦🇷 Top 10 países más similares a Argentina (ARG):\n")
top_similar = argentina_valid.nsmallest(10, 'dtw_distance')
for idx, row in top_similar.iterrows():
    print(f"   {row['country2']:10s} - DTW: {row['dtw_distance']:8.2f} | {row['overlap_years']:2.0f} años superpuestos")

print(f"\n✅ Cache DTW generado exitosamente!")
print(f"📊 Total de comparaciones con Argentina:")
print(f"   - Con superposición: {len(argentina_valid)}")
print(f"   - Sin superposición: {len(argentina_comparisons) - len(argentina_valid)}")

# Mostrar rango de años de Argentina
if 'ARG' in country_years:
    arg_years = country_years['ARG']
    print(f"\n📅 Argentina tiene datos: {arg_years[0]} - {arg_years[-1]} ({len(arg_years)} años)")


🇦🇷 Top 10 países más similares a Argentina (ARG):

   URY        - DTW:     1.38 | 33 años superpuestos
   FJI        - DTW:     1.46 | 33 años superpuestos
   BRA        - DTW:     1.50 | 33 años superpuestos
   CYP        - DTW:     1.66 | 33 años superpuestos
   COL        - DTW:     1.66 | 33 años superpuestos
   TUR        - DTW:     1.70 | 33 años superpuestos
   ITA        - DTW:     1.71 | 33 años superpuestos
   PRT        - DTW:     1.74 | 33 años superpuestos
   TUN        - DTW:     1.76 | 33 años superpuestos
   KNA        - DTW:     1.80 | 33 años superpuestos

✅ Cache DTW generado exitosamente!
📊 Total de comparaciones con Argentina:
   - Con superposición: 140
   - Sin superposición: 0

📅 Argentina tiene datos: 1989 - 2021 (33 años)
